In [178]:
import scipy.stats as sps
from sklearn.model_selection import train_test_split
import gc
from sklearn.linear_model import LinearRegression
from dateutil.parser import *
import pandas as pd
import numpy as np
import os
import yt.wrapper as yt
from collections import Counter
import json
import typing as tp
import ast
from datetime import datetime
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import operator
import re
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')

In [182]:
df = lib.execute_query("""SELECT
    cube_info.*,
    arrayFilter(x -> x != '',
            [is_20_pct_from_50k, is_7_pct_from_150k, is_4_weeks_from_grant_start]
        ) as Tags,
    toInt64(NOW()) as Timestamp
FROM (
    SELECT
        billing_account_id as Billing_account_id,
        first_name as First_name,
        last_name as Last_name,
        account_name as Account_name,
        phone as Phone_1,
        multiIf(
            user_settings_email LIKE '%@yandex.%' OR user_settings_email LIKE '%@ya.%',
            CONCAT(
                lower(replaceAll(splitByString('@',assumeNotNull(user_settings_email))[1], '.', '-')), 
                '@yandex.ru'
            ), 
            user_settings_email == 'fake@fake.ru', '',
            lower(user_settings_email)) as Email,
        'New' as Status,
        'moiseeva-m' as Assigned_to,
        'isv' as Lead_Source,
        'Cloud_Boost_Onboarding' as Lead_Source_Description
    FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
    WHERE is_isv == 1
    AND event == 'ba_created'
) as cube_info
INNER JOIN (
    SELECT
        DISTINCT
        billing_account_id,
        SUM(if (initial_amount in (50000, 150000), consumed_amount, 0)) as all_grant_consumed,
        max(if (initial_amount == 50000 and consumed_amount > 0.2 * initial_amount, 
        '50k threshold', '')) as is_20_pct_from_50k,
        max(if (initial_amount == 150000 and consumed_amount > 0.07 * initial_amount,
        '150k threshold', '')) as is_7_pct_from_150k,
        max(if (initial_amount in (50000, 150000) and toDate(NOW()) - toDate(start_time) > 4 * 7 
                                                  and toDate(NOW()) - toDate(start_time) < 5 * 7
                                                  , '4 weeks threshold', '')) 
                                                  as is_4_weeks_from_grant_start_raw,
        if (all_grant_consumed > 500, is_4_weeks_from_grant_start_raw, '') as is_4_weeks_from_grant_start,
        
        max(if (initial_amount in (50000, 150000) and toDate(NOW()) < toDate(end_time), 1, 0)) as is_active_trial
    FROM "//home/cloud_analytics/import/billing/grants_spending"
    GROUP BY billing_account_id
    HAVING length(is_4_weeks_from_grant_start || is_20_pct_from_50k || is_7_pct_from_150k) > 0
    and is_active_trial == 1
) as grant_info
ON cube_info.Billing_account_id == grant_info.billing_account_id
WHERE Phone_1 != ''
AND isNotNull(Phone_1)
FORMAT TabSeparatedWithNames
""")

In [167]:
df['Email'] = df['Email'].apply(lambda x: '["' + x + '"]' if not pd.isnull(x) else np.nan)
df['Tags'] = df['Tags'].astype(str).apply(lambda x: x.replace("'", '"') if not pd.isnull(x) else np.nan)
df['Phone_1'] = df['Phone_1'].astype(str)

In [177]:
df.shape

(0, 12)

In [172]:
drop_req = f"""
SELECT DISTINCT Billing_account_id 
from "//home/cloud_analytics/lunin-dv/call_center_output_streaming_output/drop_billing_id_isv_onboarding"
FORMAT TabSeparatedWithNames
"""
drop_df = lib.execute_query(drop_req)
df = df[~df['Billing_account_id'].isin(drop_df['Billing_account_id'])]

In [173]:
df['Billing_account_id'] =\
df['Billing_account_id'].apply(lambda x: '["' + x + '"]' if not pd.isnull(x) else np.nan)
df = df[~df['Billing_account_id'].isin(drop_df['Billing_account_id'])]

In [163]:
lib.save_table("update_leads", 
               "//home/cloud_analytics/export/crm/update_call_center_leads",
               df, append=True)

## dropped list update

In [164]:
new_drop_list_df = pd.DataFrame(df['Billing_account_id'])
lib.save_table('drop_billing_id_isv_onboarding', 
               '//home/cloud_analytics/lunin-dv/call_center_output_streaming_output',
               new_drop_list_df, append=True)